Dependencies

In [9]:
import SimpleITK as sitk
import natsort
import pathlib
import os
import shutil
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split

Paths

In [3]:
path_prefix = "/home/kanthoulis/spider/dataset/"

full_image_dir = pathlib.Path(path_prefix + "images_series")
full_label_dir = pathlib.Path(path_prefix + "labels_series")
#Directories to copy to
#train_image_path = pathlib.Path(r"D:/Spider Data/train_images")
#train_label_path = pathlib.Path(r"D:/Spider Data/train_labels")
#test_image_path = pathlib.Path(r"D:/Spider Data/test_images")
#test_label_path = pathlib.Path(r"D:/Spider Data/test_labels")

#laptop
train_image_path = pathlib.Path(path_prefix + "train_images")
train_label_path = pathlib.Path(path_prefix + "train_labels")
test_image_path = pathlib.Path(path_prefix + "test_images")
test_label_path = pathlib.Path(path_prefix + "test_labels")

full_image_dir_list = os.listdir(full_image_dir)
full_label_dir_list = os.listdir(full_label_dir)

print("full image dir len", len(full_image_dir_list))

full image dir len 380


Get patient ID from filename <br>
Last number of file name is patient series

In [7]:
def get_patient_id(filename):
    return filename.split('_')[0]


80-20 split of the images <br>
<b> Note: </b> We are applying the dataset split now instead of after extracting the 2d slices from the MRI files to ensure <br>
each patient series only belongs in either training or test data, not both 

In [8]:
# Get list of image filenames
image_files = os.listdir(full_image_dir)

# Split filenames into train and validation sets based on patient IDs
patient_ids = [get_patient_id(filename) for filename in image_files]
unique_patient_ids = list(set(patient_ids))
train_patient_ids, valid_patient_ids = train_test_split(unique_patient_ids, test_size=0.2, random_state=46)

Copy the split to separate directories

In [11]:
# Copy images and masks to train or test directories
for filename in tqdm(image_files, desc="Copying images and masks", unit="file"):
    patient_id = get_patient_id(filename)
    is_train = patient_id in train_patient_ids
    source_image_path = os.path.join(full_image_dir, filename)
    source_mask_path = os.path.join(full_label_dir, filename)  # Assuming mask filenames are same as image filenames

    if is_train:
        destination_image_path = os.path.join(train_image_path, filename)
        destination_mask_path = os.path.join(train_label_path, filename)
    else:
        destination_image_path = os.path.join(test_image_path, filename)
        destination_mask_path = os.path.join(test_label_path, filename)

    shutil.copy(source_image_path, destination_image_path)
    shutil.copy(source_mask_path, destination_mask_path)

Copying images and masks: 100%|██████████| 380/380 [00:10<00:00, 37.66file/s]


Delete original directories for cleanup

In [12]:
#Delete folders after processing to avoid cluttering disk space
if full_image_dir.exists() and full_image_dir.is_dir():
    shutil.rmtree(full_image_dir)
    print(f"Deleted folder: {full_image_dir}")
else:
    print(f"Folder not found or already deleted: {full_image_dir}")

if full_label_dir.exists() and full_label_dir.is_dir():
    shutil.rmtree(full_label_dir)
    print(f"Deleted folder: {full_label_dir}")
else:
    print(f"Folder not found or already deleted: {full_label_dir}")

Deleted folder: /home/kanthoulis/spider/dataset/images_series
Deleted folder: /home/kanthoulis/spider/dataset/labels_series
